In [ ]:
!pip install langchain pyarrow chromadb tiktoken tabulate sqlalchemy sqlalchemy-bigquer google-cloud-aiplatform google-cloud-bigquery google-cloud-bigquery-storage

需要配置 GOOGLE_APPLICATION_CREDENTIALS 环境变量
export GOOGLE_APPLICATION_CREDENTIALS=/home/user/code/sa.json

In [ ]:
!export GOOGLE_APPLICATION_CREDENTIALS=/home/user/code/sa.json

重启 Kernel

测试是否可以调用Palm2 API

In [ ]:
from langchain.llms import VertexAI
from langchain import PromptTemplate, LLMChain

template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

llm = VertexAI()

llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "What NFL team won the Super Bowl in the year Justin Beiber was born?"

llm_chain.run(question)

创建测试的BQ 表格

In [ ]:
PROJECT_ID = "fine-acronym-336109"  # @param {type:"string"} ## Update this with your project id
LOCATION = "us-central1" # @param {type:"string"} ## Continue with us-central1
DATASET = "iowa_liquor_sales"

In [ ]:
from google.cloud import bigquery
from google.cloud.bigquery import Client
client = Client(project=PROJECT_ID)
dataset_id = DATASET

In [ ]:
query = """
CREATE SCHEMA `{PROJECT_ID}.{dataset_id}`
OPTIONS(
  location="us"
  )
""".format(
    PROJECT_ID=PROJECT_ID, dataset_id=dataset_id
)
query_job = client.query(query)
print(query_job.result())

In [ ]:
query = """
create or replace table `{PROJECT_ID}.{dataset_id}.sales`
as
select invoice_and_item_number,
date,
store_number,
store_name,
address,
city,
zip_code,
county_number,
county,
category,
category_name,
vendor_number,
vendor_name,
item_number,
item_description,
pack,
bottle_volume_ml,
state_bottle_cost,
state_bottle_retail,
bottles_sold,
sale_dollars,
volume_sold_liters,
volume_sold_gallons
from `bigquery-public-data.iowa_liquor_sales.sales`
""".format(
    PROJECT_ID=PROJECT_ID, dataset_id=dataset_id
)
query_job = client.query(query)
print(query_job.result())

测试LLM with BQ

In [ ]:
from google.cloud import bigquery
from sqlalchemy import *
from sqlalchemy.engine import create_engine
from sqlalchemy.schema import *
import os
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.agents import AgentExecutor

service_account_file = "/home/user/code/sa.json" # Change to where your service account key file is located
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = service_account_file

project = PROJECT_ID
dataset = DATASET
table = "sales"
sqlalchemy_url = f'bigquery://{project}/{dataset}?credentials_path={service_account_file}'

In [21]:
from langchain.llms import VertexAI
from langchain import PromptTemplate, LLMChain

llm = VertexAI()
db = SQLDatabase.from_uri(sqlalchemy_url)

toolkit = SQLDatabaseToolkit(db=db, llm=llm)
agent_executor = create_sql_agent(
llm=llm,
toolkit=toolkit,
# verbose=True,
top_k=40,
)

def query(question_str):
    return agent_executor.run(question_str)


In [19]:
query("Count total number of invoices")




> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: sales
Thought:The sales table seems relevant. I should query the schema of the sales table.
Action: sql_db_schema
Action Input: sales
Observation: 
CREATE TABLE `sales` (
	`invoice_and_item_number` STRING, 
	`date` DATE, 
	`store_number` STRING, 
	`store_name` STRING, 
	`address` STRING, 
	`city` STRING, 
	`zip_code` STRING, 
	`county_number` STRING, 
	`county` STRING, 
	`category` STRING, 
	`category_name` STRING, 
	`vendor_number` STRING, 
	`vendor_name` STRING, 
	`item_number` STRING, 
	`item_description` STRING, 
	`pack` INT64, 
	`bottle_volume_ml` INT64, 
	`state_bottle_cost` FLOAT64, 
	`state_bottle_retail` FLOAT64, 
	`bottles_sold` INT64, 
	`sale_dollars` FLOAT64, 
	`volume_sold_liters` FLOAT64, 
	`volume_sold_gallons` FLOAT64
)

/*
3 rows from sales table:
invoice_and_item_number	date	store_number	store_name	address	city	zip_code	county_number	county	category	category_name	vendor_numb

'The total number of invoices is 26592863.'

In [16]:
query("what were the total sales for County DALLAS in 2022?")


'12992381.990000008'

In [ ]:
query("Can you identify the stores with the highest number of sales? Please provide the store names along with their zip codes , dollar sales and city names.")


In [ ]:
!pip install gradio

In [22]:
import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown(
    """
    ## Ask BigQuery

    This demo is to showcase answering questions on a tabular data available in Big Query using Vertex PALM LLM & Langchain.

    Dataset: Public dataset on Iowa Liquor Sales

    Dataset dimensions: ~26 million rows and 24 columns



    ### Sample Inputs:
    1. Count total number of invoices
    2. Identify the top stores along with their zipcode & city name in terms of Number of Sales
    3. Write an SQL query to extract the month from the date field, calculate the sum of monthly sales for "GLENLIVET 12YR" in 2023, and display the results as month name followed by sales for each month in 2023.

    ### Enter a search query...

    """)
    with gr.Row():
      with gr.Column():
        input_text = gr.Textbox(label="Question", placeholder="Count total number of invoices")

    with gr.Row():
      generate = gr.Button("Ask BigQuery")

    with gr.Row():
      label2 = gr.Textbox(label="Output")
    # with gr.Row():
    #   label3 = gr.Textbox(label="SQL query generated by LLM")

    generate.click(fn=query,inputs=input_text, outputs=label2)
demo.launch(share=True, debug=False)

Running on local URL:  http://127.0.0.1:7864
Running on public URL: https://7b8845545dd95492ac.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
